In [1]:
#pip install NRCLex

In [2]:
# Import required modules
import nltk
nltk.download('punkt')
import pandas as pd
import numpy as np
from nltk import word_tokenize
from nltk.stem.snowball import SnowballStemmer
import requests
from LeXmo import LeXmo
import pickle
import os
from tqdm.notebook import tqdm
import statsmodels.formula.api as sm
from scipy.stats import t 

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/ellietupin/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/ellietupin/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


# Parameters

In [3]:
sp = "sp500"

In [4]:
df_path = "../data/earning_call_dataframe/df_final_"+sp 
# df_path = "data/df_final"
with open(df_path,"rb") as file :
    df_final = pickle.load(file)
display(df_final)

,id,date,ticker,text,earning_call_quarter
0,4264085,2019-05-14,A,Question-and-Answer Session\n \nOperator\n \n[...,2019Q1
1,4285738,2019-08-14,A,Question-and-Answer Session\n \nOperator\n \n[...,2019Q2
2,4309111,2019-11-26,A,Question-and-Answer Session\n \nOperator\n \n[...,2019Q3
3,4325101,2020-02-18,A,Question-and-Answer Session\n \nOperator\n \n[...,2019Q4
4,4349475,2020-05-21,A,Question-and-Answer Session\n \nOperator\n \n[...,2020Q1
...,...,...,...,...,...
1855,4422752,2021-04-30,DRE,Question-and-Answer Session\n \nOperator\n \n[...,2021Q1
1856,4442846,2021-07-29,DRE,Question-and-Answer Session\n \nOperator\n \nT...,2021Q2
1857,4463265,2021-10-28,DRE,Question-and-Answer Session\n \nOperator\n \nT...,2021Q3
1858,4482287,2022-01-27,DRE,Question-and-Answer Session \n \nOperator\n \n...,2021Q4


# Import datas from yahoo finance

Import html tickers from data/ticker
dir_path = "data/ticker/"
list_tickers = os.listdir(dir_path)
list_tickers.remove('.DS_Store') # for mac
list_tickers.remove('id_to_date')
print(list_tickers)

Test with few stocks
list_tickers = ['AAPL']

In [5]:
# Get the month from which we import stock returns :
    # Q1 : from january (1) to march (3)
    # Q2 : from april (4) to june (6)
    # Q3 : from july (7) to septmber (9)
    # Q4 : from october (10) to december (12)

#df_final['earning_call_quarter'] = pd.to_datetime(df_final['earning_call_quarter'])
#df_final['date'] = pd.to_datetime(df_final['date'])

# Quarters start and end
quarter_start = df_final['earning_call_quarter'].iloc[0].quarter
quarter_end = df_final['earning_call_quarter'].iloc[-1].quarter + 1 # need 1 additionnal quarter to regress future chang in market cap

# Months start and end
month_start = quarter_start*2-1
month_end = quarter_end*3

# Day end : 30 or 31 ?
if month_end in [3,12]:
    day_end = 31
else:
    day_end = 30


In [6]:
import pandas as pd
import yfinance as yf
import datetime
import time
import requests
import io

# Take a margin of 1 year to calculate abnormal returns
start = datetime.datetime(df_final['earning_call_quarter'].iloc[0].year,month_start,2)
end   = datetime.datetime(df_final['earning_call_quarter'].iloc[-1].year,month_end,day_end)

# Create empty dataframe
stock_final = pd.DataFrame()

# Iterate over each symbol
list_tickers = list(set(list(df_final['ticker'])))
for i in list_tickers:  
    
    # print the symbol which is being downloaded
    print( str(list_tickers.index(i)) + str(' : ') + i, sep=',', end=',', flush=True)  
    
    try:
        # download the stock price 
        stock = []
        stock = yf.download(i,start=start, end=end, progress=False)
        
        # append the individual stock prices 
        if len(stock) == 0:
            None
        else:
            stock['ticker']=i
            stock_final = stock_final.append(stock,sort=False)
    except Exception:
        None
        
display(stock_final)

0 : BDX,1 : ADP,2 : ALK,3 : CDW,4 : ADSK,5 : ANTM,6 : AOS,7 : AEP,8 : BLK,9 : ABT,10 : BR,11 : ARE,12 : DLTR,13 : CF,14 : CLX,15 : AMAT,16 : COF,17 : CTSH,18 : CI,19 : BSX,20 : DG,21 : DRE,22 : ATO,23 : AVB,24 : DOV,25 : AFL,26 : DE,27 : DHR,28 : CAH,29 : AAL,30 : AON,31 : CMS,32 : BKNG,33 : CMI,34 : AEE,35 : BBWI,36 : CE,37 : CAT,38 : ADM,39 : CBRE,40 : CPT,41 : BAC,42 : DFS,43 : BRO,44 : DGX,45 : CHD,46 : BEN,47 : APTV,48 : BALL,49 : BA,50 : DAL,51 : BXP,52 : CL,53 : BIIB,54 : CZR,55 : D,56 : ANET,57 : ANSS,58 : CMG,59 : CVS,60 : DLR,61 : BBY,62 : ALGN,63 : APA,64 : APH,65 : AMZN,66 : DHI,67 : AMP,68 : BMY,69 : CDAY,70 : AZO,71 : CSX,72 : ALL,73 : CME,74 : CINF,75 : AES,76 : AIG,77 : DD,78 : DIS,79 : BK,80 : ABBV,81 : CHTR,82 : C,83 : AKAM,84 : ABMD,85 : CHRW,86 : APD,87 : AME,88 : CMA,89 : AVY,90 : A,91 : CDNS,92 : AAPL,93 : ALB,94 : AXP,95 : CB,96 : CBOE,97 : AIZ,98 : CCI,99 : BKR,

,Open,High,Low,Close,Adj Close,Volume,ticker
Date,,,,,,,
2019-01-02,216.556091,217.570724,212.897568,214.712189,205.051193,1206220,BDX
2019-01-03,212.770737,213.414627,205.580490,206.048782,196.777618,1937250,BDX
2019-01-04,208.097565,213.258530,208.097565,211.697556,202.172241,2024170,BDX
2019-01-07,211.531708,217.307312,211.063416,213.307312,203.709549,1702013,BDX
2019-01-08,214.848785,217.570724,214.487808,217.268295,207.492310,1160300,BDX
...,...,...,...,...,...,...,...
2022-06-13,34.320000,34.389999,32.849998,33.299999,33.299999,13025700,BKR
2022-06-14,34.240002,34.500000,32.520000,32.930000,32.930000,9909100,BKR
2022-06-15,33.189999,33.189999,31.299999,31.740000,31.740000,13069200,BKR


In [7]:
# Add quarter column
stock_final['date']=stock_final.index
stock_final['quarter'] = stock_final['date'].dt.to_period('Q')
# Calculate market capitalization
stock_final['market_cap']=stock_final['Adj Close']*stock_final['Volume']
# Drop useless columns
stock_final=stock_final.drop(columns=['Open', 'High','Low','Close','date','Adj Close','Volume'])

# Sum of 100 stocks for each quarter
stock_final = stock_final.groupby(['quarter']).sum()
stock_final['percent_change']=stock_final['market_cap'].pct_change()

stock_final = stock_final.iloc[1:,:]  # remove first row
stock_final.index = stock_final.index.to_timestamp()

# Shift change in market cap to regress the future change in market cap on last period change in emotions
stock_final['percent_change']=stock_final['percent_change'].shift(periods=-1)
# Remove last row
stock_final = stock_final.iloc[:-1]

display(stock_final)

,market_cap,percent_change
quarter,,
2019-04-01,2.359787e+12,-0.059002
2019-07-01,2.220554e+12,0.059203
2019-10-01,2.352018e+12,0.582558
2020-01-01,3.722204e+12,0.044402
2020-04-01,3.887478e+12,0.059278
2020-07-01,4.117920e+12,-0.078061
2020-10-01,3.796469e+12,0.009812
2021-01-01,3.833721e+12,-0.083499
2021-04-01,3.513609e+12,-0.007954


# Text analysis

In [8]:
texts = df_final['text']
n = len(texts)

In [9]:
#N=len(texts)

for i in range(n):
    # creating objects
    text=texts[i]
    emo = LeXmo.LeXmo(text)
    emo.pop('text', None)
    emo_array = np.array(list(emo.items()))
    score = emo_array[:,1]
    if i==0:
        con = score
    else :
        # concatenate all score in a big list
        con = np.vstack((con, score))

In [10]:
# Create dataframe
score = np.reshape(con, (n, 10))
column=['anger','anticipation','disgust','fear','joy','negative','positive','sadness','surprise','trust']
df = pd.DataFrame(score, columns = column)
df[column]=df[column].astype(float)

# Earning call informations
df['id'] = df_final['id']
df['quarter'] = df_final['earning_call_quarter']
df['ticker'] = df_final['ticker']

# Calculate change in sentiments from one quarter to the other
df_change = df
df_change[column] = df[column].pct_change()
# Mean over each quarter
df_change = df_change.groupby(['quarter']).mean()
# Remove first row
df_change=df_change.iloc[1:,:]
# Remplace Nan with zeros
df_change = df_change.fillna(0)
df_change.index = (df_change.index).to_timestamp()
display(df_change)

# Merge dataframes
df_ols = pd.merge(stock_final, df_change, left_index=True, right_index=True)
display(df_ols)

,anger,anticipation,disgust,fear,joy,negative,positive,sadness,surprise,trust
quarter,,,,,,,,,,
2019-04-01,-0.026106,0.172643,0.151611,-0.324919,0.377327,-0.083514,-0.022175,-0.103786,0.261362,0.064781
2019-07-01,-0.009045,0.029816,-0.459479,0.081042,0.243965,0.146399,0.218529,0.376327,-0.036194,0.141862
2019-10-01,0.108000,0.085701,1.105201,1.192917,-0.295293,0.195326,-0.183799,0.136808,0.260155,-0.248805
2020-01-01,0.952888,-0.373987,0.115936,-0.062614,-0.110076,0.249182,-0.071187,0.074605,-0.487982,-0.042499
2020-04-01,-0.279295,0.333088,-0.637577,-0.295863,0.201175,-0.246159,0.064171,-0.063275,0.293573,-0.030061
2020-07-01,-0.383154,-0.141768,1.525441,-0.442918,0.112742,0.165588,0.153083,0.089848,0.124837,0.289876
2020-10-01,-0.275954,-0.012004,-0.447683,0.052033,-0.032587,-0.316179,-0.091975,-0.249066,-0.144799,-0.078087
2021-01-01,0.546988,0.294170,0.117269,0.675903,0.081611,-0.011647,0.180931,0.214423,0.017513,-0.033713
2021-04-01,-0.425111,-0.113414,-0.731221,-0.435563,-0.235012,0.042981,-0.206340,-0.021643,0.143630,-0.165104


,market_cap,percent_change,anger,anticipation,disgust,fear,joy,negative,positive,sadness,surprise,trust
quarter,,,,,,,,,,,,
2019-04-01,2.359787e+12,-0.059002,-0.026106,0.172643,0.151611,-0.324919,0.377327,-0.083514,-0.022175,-0.103786,0.261362,0.064781
2019-07-01,2.220554e+12,0.059203,-0.009045,0.029816,-0.459479,0.081042,0.243965,0.146399,0.218529,0.376327,-0.036194,0.141862
2019-10-01,2.352018e+12,0.582558,0.108000,0.085701,1.105201,1.192917,-0.295293,0.195326,-0.183799,0.136808,0.260155,-0.248805
2020-01-01,3.722204e+12,0.044402,0.952888,-0.373987,0.115936,-0.062614,-0.110076,0.249182,-0.071187,0.074605,-0.487982,-0.042499
2020-04-01,3.887478e+12,0.059278,-0.279295,0.333088,-0.637577,-0.295863,0.201175,-0.246159,0.064171,-0.063275,0.293573,-0.030061
2020-07-01,4.117920e+12,-0.078061,-0.383154,-0.141768,1.525441,-0.442918,0.112742,0.165588,0.153083,0.089848,0.124837,0.289876
2020-10-01,3.796469e+12,0.009812,-0.275954,-0.012004,-0.447683,0.052033,-0.032587,-0.316179,-0.091975,-0.249066,-0.144799,-0.078087
2021-01-01,3.833721e+12,-0.083499,0.546988,0.294170,0.117269,0.675903,0.081611,-0.011647,0.180931,0.214423,0.017513,-0.033713
2021-04-01,3.513609e+12,-0.007954,-0.425111,-0.113414,-0.731221,-0.435563,-0.235012,0.042981,-0.206340,-0.021643,0.143630,-0.165104


# OLS

In [11]:
# ols
result = sm.ols(formula="percent_change ~ anger+anticipation+disgust+fear+joy+negative+positive+sadness+surprise+trust", data=df_ols).fit()
result.summary()

/opt/anaconda3/lib/python3.9/site-packages/scipy/stats/stats.py:1541: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=12
  warnings.warn("kurtosistest only valid for n>=20 ... continuing "


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:         percent_change   R-squared:                       0.860
Model:                            OLS   Adj. R-squared:                 -0.539
Method:                 Least Squares   F-statistic:                    0.6150
Date:                Sun, 19 Jun 2022   Prob (F-statistic):              0.769
Time:                        10:06:01   Log-Likelihood:                 14.951
No. Observations:                  12   AIC:                            -7.903
Df Residuals:                       1   BIC:                            -2.569
Df Model:                          10                                         
Covariance Type:            nonrobust                                         
================================================================================
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
Intercept        0.0284      0.134      0.212      0.867      -1.675       1.732
anger           -0.7843      1.020     -0.769      0.583     -13.744      12.175
anticipation     0.3001      2.206      0.136      0.914     -27.732      28.332
disgust          0.0194      0.270      0.072      0.954      -3.413       3.451
fear             0.3006      0.326      0.921      0.526      -3.847       4.448
joy              1.3137      2.792      0.471      0.720     -34.162      36.790
negative         2.6956      2.664      1.012      0.496     -31.148      36.539
positive         1.6778      5.686      0.295      0.817     -70.568      73.924
sadness         -1.7378      1.991     -0.873      0.543     -27.032      23.556
surprise        -0.8187      1.021     -0.802      0.570     -13.791      12.153
trust           -3.1376      8.094     -0.388      0.765    -105.978      99.703
==============================================================================
Omnibus:                        1.302   Durbin-Watson:                   0.575
Prob(Omnibus):                  0.522   Jarque-Bera (JB):                1.025
Skew:                           0.563   Prob(JB):                        0.599
Kurtosis:                       2.117   Cond. No.                         190.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""